# Datos estructurados: Arrays estructurados de NumPy

Aunque a menudo nuestros datos pueden ser bien representados por un array homogéneo de valores, a veces no es el caso. Esta sección demuestra el uso de los *arrays estructurados* y los *arrays de registros* de NumPy, que proporcionan un almacenamiento eficiente para datos compuestos y heterogéneos.  Mientras que los patrones mostrados aquí son útiles para operaciones simples, escenarios como este se prestan a menudo al uso de Pandas ``Dataframe``, que exploraremos en el [Capítulo 3](03.00-Introducción a Pandas.ipynb).

In [ ]:
import numpy as np

Imagina que tenemos varias categorías de datos sobre una serie de personas (digamos, nombre, edad y peso), y nos gustaría almacenar estos valores para utilizarlos en un programa de Python.
Sería posible almacenarlos en tres arrays distintos:

In [ ]:
name = ['Alice', 'Bob', 'Cathy', 'Doug']
age = [25, 45, 37, 19]
weight = [55.0, 85.5, 68.0, 61.5]

Pero esto es un poco torpe. No hay nada aquí que nos diga que los tres arrays están relacionados; sería más natural si pudiéramos usar una sola estructura para almacenar todos estos datos.
NumPy puede manejar esto a través de arrays estructurados, que son arrays con tipos de datos compuestos.

Recordemos que anteriormente creamos un array simple usando una expresión como esta:

In [ ]:
x = np.zeros(4, dtype=int)

De forma similar, podemos crear un array estructurado utilizando una especificación de tipo de datos compuesto:

In [ ]:
# Utilizar un tipo de datos compuesto para arrays estructurados
data = np.zeros(4, dtype={'names':('name', 'age', 'weight'),
                          'formats':('U10', 'i4', 'f8')})
print(data.dtype)

[('name', '<U10'), ('age', '<i4'), ('weight', '<f8')]


Aquí ``'U10'`` se traduce como "string Unicode de longitud máxima 10", ``'i4'`` se traduce como "integer de 4 bytes (es decir, 32 bits)", y ``'f8'`` se traduce como "float de 8 bytes (es decir, 64 bits)".
En la siguiente sección hablaremos de otras opciones para estos códigos de tipo.

Ahora que hemos creado un array contenedor vacío, podemos llenar el array con nuestros lists de valores:

In [ ]:
data['name'] = name
data['age'] = age
data['weight'] = weight
print(data)

[('Alice', 25, 55.0) ('Bob', 45, 85.5) ('Cathy', 37, 68.0)
 ('Doug', 19, 61.5)]


Tal y como esperábamos, los datos están ahora organizados en un único bloque de memoria.

Lo práctico de los arrays estructuradas es que ahora puedes referirte a los valores por índice o por nombre:

In [ ]:
# Obtener todos los nombres
data['name']

array(['Alice', 'Bob', 'Cathy', 'Doug'], 
      dtype='<U10')

In [ ]:
# Obtener la primera fila de datos
data[0]

('Alice', 25, 55.0)

In [ ]:
# Obtener el nombre de la última fila
data[-1]['name']

'Doug'

Utilizando el enmascaramiento booleano, esto permite incluso realizar algunas operaciones más sofisticadas, como el filtrado por edad:

In [ ]:
# Get names where age is under 30
data[data['age'] < 30]['name']

array(['Alice', 'Doug'], 
      dtype='<U10')

Ten en cuenta que si quieres hacer operaciones más complicadas que éstas, probablemente deberías considerar el paquete Pandas, que se trata en el siguiente capítulo.
Como veremos, Pandas proporciona un objeto ``Dataframe``, que es una estructura construida sobre arrays de NumPy que ofrece una variedad de funcionalidades útiles de manipulación de datos similares a las que hemos mostrado aquí, así como mucho, mucho más.

## Creación de arrays estructuradas

Los tipos de datos de los arrays estructurados se pueden especificar de varias maneras.
Anteriormente, vimos el método del diccionario:

In [ ]:
np.dtype({'names':('name', 'age', 'weight'),
          'formats':('U10', 'i4', 'f8')})

dtype([('name', '<U10'), ('age', '<i4'), ('weight', '<f8')])

Para mayor claridad, los tipos numéricos pueden ser especificados usando tipos de Python o ``dtype`` de NumPy en su lugar:

In [ ]:
np.dtype({'names':('name', 'age', 'weight'),
          'formats':((np.str_, 10), int, np.float32)})

dtype([('name', '<U10'), ('age', '<i8'), ('weight', '<f4')])

Un tipo compuesto también puede especificarse como un list de tuplas:

In [ ]:
np.dtype([('name', 'S10'), ('age', 'i4'), ('weight', 'f8')])

dtype([('name', 'S10'), ('age', '<i4'), ('weight', '<f8')])

Si los nombres de los tipos no le importan, puede especificar los tipos solos en un string separada por comas:

In [ ]:
np.dtype('S10,i4,f8')

dtype([('f0', 'S10'), ('f1', '<i4'), ('f2', '<f8')])

Los códigos de formato string abreviados pueden parecer confusos, pero se basan en principios sencillos.
El primer carácter (opcional) es ``<`` o ``>``, que significa "little endian" o "big endian", respectivamente, y especifica la convención de ordenación de los bits significativos.
El siguiente carácter especifica el tipo de datos: caracteres, bytes, ints, puntos flotantes, etc. (véase la tabla siguiente).
El último carácter o caracteres representan el tamaño del objeto en bytes.

| Carácter | Descripción | Ejemplo                            |
| ---------        | -----------           | -------                             | 
| ``'b'``          | Byte                  | ``np.dtype('b')``                   |
| ``'i'``          | Integer con signo        | ``np.dtype('i4') == np.int32``      |
| ``'u'``          | Integer sin signo      | ``np.dtype('u1') == np.uint8``      |
| ``'f'``          | Punto flotante        | ``np.dtype('f8') == np.int64``      |
| ``'c'``          | Punto flotante complejo| ``np.dtype('c16') == np.complex128``|
| ``'S'``, ``'a'`` | String                | ``np.dtype('S5')``                  |
| ``'U'``          | String Unicode        | ``np.dtype('U') == np.str_``        |
| ``'V'``          | Datos sin procesar (void)      | ``np.dtype('V') == np.void``        |

## Tipos de compuestos más avanzados

Es posible definir tipos compuestos aún más avanzados.
Por ejemplo, se puede crear un tipo en el que cada elemento contenga un array o matriz de valores.
Aquí, crearemos un tipo de datos con un componente ``mat`` que consiste en un array de punto float de $3 \times 3$:

In [ ]:
tp = np.dtype([('id', 'i8'), ('mat', 'f8', (3, 3))])
X = np.zeros(1, dtype=tp)
print(X[0])
print(X['mat'][0])

(0, [[0.0, 0.0, 0.0], [0.0, 0.0, 0.0], [0.0, 0.0, 0.0]])
[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]


Ahora cada elemento del array ``X`` consiste en un ``id`` y un array de $3 \times 3$.
¿Por qué usar esto en lugar de un simple array multidimensional, o quizás un diccionario de Python?
La razón es que este ``dtype`` de NumPy se mapea directamente en una definición de estructura C, por lo que el buffer que contiene el contenido del array puede ser accedido directamente dentro de un programa C escrito apropiadamente.
Si te encuentras escribiendo una interfaz de Python para una biblioteca heredada de C o Fortran que manipula datos estructurados, probablemente encontrarás arrays estructurados bastante útiles.

## RecordArrays: Arrays estructuradas con un giro

NumPy también proporciona la clase ``np.recarray``, que es casi idéntica a los arrays estructuradas que acabamos de describir, pero con una característica adicional: se puede acceder a los campos como atributos en lugar de como claves de diccionario.
Recordemos que antes accedíamos a las edades escribiendo

In [ ]:
data['age']

array([25, 45, 37, 19], dtype=int32)

Si, en cambio, vemos nuestros datos como un array de registros, podemos acceder a ellos con algo menos de pulsaciones:

In [ ]:
data_rec = data.view(np.recarray)
data_rec.age

array([25, 45, 37, 19], dtype=int32)

La desventaja es que, en el caso de los arrays de registros, hay una sobrecarga adicional en el acceso a los campos, incluso cuando se utiliza la misma sintaxis. Podemos ver esto aquí:

In [ ]:
%timeit data['age']
%timeit data_rec['age']
%timeit data_rec.age

1000000 loops, best of 3: 241 ns per loop
100000 loops, best of 3: 4.61 µs per loop
100000 loops, best of 3: 7.27 µs per loop


El hecho de que la notación más cómoda merezca la pena por la sobrecarga adicional dependerá de su propia aplicación.

## En Pandas

Esta sección sobre arrays estructurados y de registros está a propósito al final de este capítulo, porque conduce muy bien al siguiente paquete que cubriremos: Pandas.
Los arrays estructurados como los que se han discutido aquí son buenos para conocer ciertas situaciones, especialmente en caso de que estés usando arrays de NumPy para mapear en formatos de datos binarios en C, Fortran u otro lenguaje.
Para el uso diario de datos estructurados, el paquete Pandas es una opción mucho mejor, y nos sumergiremos en una discusión completa de la misma en el capítulo siguiente.